In [ ]:
!pip install pandas
!pip install geonamescache
!pip install plotly
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import pandas as pd

import selenium.webdriver as wd
from selenium.webdriver.common.by import By
import time

import geonamescache

from ipywidgets import widgets

import plotly.express as px

In [ ]:
bar = widgets.IntProgress(
        min=0,
        max=11,
        description='Loading:',
        style={'bar_color': 'gray'}
    )

text = widgets.Text(placeholder='Mumbai', description='City: ')

submit_button = widgets.Button(description='Submit')

In [ ]:
def getWeather(city):
    dfs = []
    for i in range(12):
        ccode = gc.search_cities(city)[0]['countrycode']
        cityname = city

        url = 'https://www.wunderground.com/history/monthly/{}/{}/date/2021-{}'.format(ccode, cityname, str(i+1).zfill(2))
        
        co = wd.ChromeOptions()
        co.add_argument('--headless')
        co.add_argument('--no-sandbox')
        co.add_argument('--disable-dev-shm-usage')
        dr = wd.Chrome('chromedriver', options=co)

        dr.get(url)

        time.sleep(10)

        a = dr.find_element(By.CSS_SELECTOR,
                            'table.days[_ngcontent-app-root-c208] tbody[_ngcontent-app-root-c208] tr[_ngcontent-app-root-c208] td[_ngcontent-app-root-c208]:nth-child(2n)')

        df = pd.DataFrame([eval(q.split()[1]) for q in a.text.split('\n')[1:]])

        dfs.append(df)

        dr.quit()
        
        bar.value = i
        
    temp = pd.concat(dfs)
        
    temp.columns = ['Temperature (°F)']

    temp.reset_index(drop=True, inplace=True)

    temp.index = pd.date_range(start='2021-01', periods=len(temp), freq='D', name='Date')
    
    return temp

In [ ]:
cityx = []
gc = geonamescache.GeonamesCache()
def on_button_click(b):
    cityx.append(text.value)
    display(bar)
    city = cityx[0]
    temp = getWeather(city.capitalize())
    display(px.line(data_frame=temp, y='Temperature (°F)', color_discrete_sequence=['grey'],
        title='{}, {} in 2021'.format(city.capitalize(), gc.search_cities(city.capitalize())[0]['countrycode'].upper())))

In [ ]:
submit_button.on_click(on_button_click)

In [ ]:
display(text, submit_button)